https://www.kaggle.com/code/takeshimorimura/14th-place-final-submission

In [6]:
import os
import random
import pandas as pd
import numpy as np
import lightgbm as lgb
# import gresearch_crypto
import datetime as dt
import gc
from pathlib import Path
import pickle

ASSET_DETAILS_CSV = 'data/asset_details.csv'
SUP_TRAIN = "data/supplemental_train.csv"
TRAIN_CSV = 'data/train.csv'
# PROC = Path("../input/20220129-process-all-data")
# MODEL = Path("../input/20220129-final-model")
# MODEL_ALL_ASSET = Path("../input/20220131-final-model-one-model")

SEED = 1

REMOVE_LB_TEST_OVERLAPPING_DATA = True

In [4]:
def fix_all_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    
fix_all_seeds(SEED)

In [7]:
df_train = pd.read_csv(TRAIN_CSV)
df_train.head()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1514764860,2,40.0,2376.5800,2399.5000,2357.1400,2374.5900,19.233005,2373.116392,-0.004218
1,1514764860,0,5.0,8.5300,8.5300,8.5300,8.5300,78.380000,8.530000,-0.014399
2,1514764860,1,229.0,13835.1940,14013.8000,13666.1100,13850.1760,31.550062,13827.062093,-0.014643
3,1514764860,5,32.0,7.6596,7.6596,7.6567,7.6576,6626.713370,7.657713,-0.013922
4,1514764860,7,5.0,25.9200,25.9200,25.8740,25.8770,121.087310,25.891363,-0.008264


In [8]:
sup_train = pd.read_csv(SUP_TRAIN)
sup_train.head()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1632182460,3,561.0,2.079028,2.080605,2.0720,2.076458,2.804627e+05,2.075869,0.003085
1,1632182460,2,169.0,541.006000,541.200000,539.7000,540.722000,1.889432e+02,540.716930,-0.000607
2,1632182460,0,400.0,363.737500,363.900000,363.0000,363.654000,8.792685e+02,363.499554,-0.019375
3,1632182460,1,1933.0,42986.143333,43001.000000,42898.0000,42947.065566,9.183861e+01,42942.976718,-0.000374
4,1632182460,4,348.0,0.208326,0.208400,0.2078,0.208200,1.051337e+06,0.208077,-0.001233


In [9]:
df_train = pd.concat((df_train, sup_train))
df_train['datetime'] = pd.to_datetime(df_train['timestamp'], unit='s')
df_train = df_train.set_index("datetime")

del sup_train
gc.collect()

df_train.head()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
datetime,,,,,,,,,,
2018-01-01 00:01:00,1514764860,2,40.0,2376.5800,2399.5000,2357.1400,2374.5900,19.233005,2373.116392,-0.004218
2018-01-01 00:01:00,1514764860,0,5.0,8.5300,8.5300,8.5300,8.5300,78.380000,8.530000,-0.014399
2018-01-01 00:01:00,1514764860,1,229.0,13835.1940,14013.8000,13666.1100,13850.1760,31.550062,13827.062093,-0.014643
2018-01-01 00:01:00,1514764860,5,32.0,7.6596,7.6596,7.6567,7.6576,6626.713370,7.657713,-0.013922
2018-01-01 00:01:00,1514764860,7,5.0,25.9200,25.9200,25.8740,25.8770,121.087310,25.891363,-0.008264


In [10]:
df_asset_details = pd.read_csv(ASSET_DETAILS_CSV).sort_values("Asset_ID")
df_asset_details

,Asset_ID,Weight,Asset_Name
1,0,4.304065,Binance Coin
2,1,6.779922,Bitcoin
0,2,2.397895,Bitcoin Cash
10,3,4.406719,Cardano
13,4,3.555348,Dogecoin
3,5,1.386294,EOS.IO
5,6,5.894403,Ethereum
4,7,2.079442,Ethereum Classic
11,8,1.098612,IOTA
6,9,2.397895,Litecoin


In [11]:
df_asset_details_asset = df_asset_details.set_index("Asset_ID")
weight = df_asset_details_asset["Weight"].to_dict()
weight

{0: 4.30406509320417,
 1: 6.779921907472252,
 2: 2.3978952727983707,
 3: 4.406719247264253,
 4: 3.555348061489413,
 5: 1.3862943611198906,
 6: 5.8944028342648505,
 7: 2.079441541679836,
 8: 1.0986122886681098,
 9: 2.3978952727983707,
 10: 1.0986122886681098,
 11: 1.6094379124341005,
 12: 2.079441541679836,
 13: 1.791759469228055}

In [12]:
def calc_sma_diff_test(close, timeperiod_short, timeperiod_long):
    res_short = close[-timeperiod_short:].mean()
    res_long = close[-timeperiod_long:].mean()
    res = (res_long - res_short) / res_long
    return res

# 計算早いけどちょっとだけ値がずれる。
def calc_bbands_test(close, timeperiod, std=2):
    close_std = close[-timeperiod:].std(ddof=0)
    rol = close[-timeperiod:].mean()
    upper = rol + close_std * std
    lower = rol - close_std * std
    res = (upper - close[-1]) / (upper - lower)
    return res

# # これ使うならcloseをSeriesで渡す必要あり
# # def calc_bbands_test(close, timeperiod, std=2):
# #     close_std = close[-timeperiod:].rolling(timeperiod).std(ddof=0)[-1]
# #     rol = close[-timeperiod:].rolling(timeperiod).mean()[-1]
# #     upper = rol + close_std * std
# #     lower = rol - close_std * std
# #     res = (upper - close[-1]) / (upper - lower)
# #     return res

def calc_rsi_test(close, timeperiod):
    # https://kabu.com/investment/guide/technical/08.html
    diff = np.diff(close[-(timeperiod+1):])
    A = diff[diff>0].sum() / timeperiod
    B = -diff[diff<0].sum() / timeperiod
    rsi = 100 * A / (A + B)
#     rsi = 100 - (100 / (1 + A/B))
    return rsi

def calc_atr_test(high, low, close, timeperiod):
    A = high[-timeperiod:] - close[-(timeperiod+1):-1]
    B = close[-(timeperiod+1):-1] - low[-timeperiod:]
    C = high[-timeperiod:] - low[-timeperiod:]
    res = np.vstack((A, B, C)).max(axis=0).mean()
    return res

def calc_natr_test(high, low, close, timeperiod):
    res = calc_atr_test(high, low, close, timeperiod) / close[-1]
    return res

def calc_minus_di_test(high, low, close, timeperiod):
    high_diff = np.diff(high[-(timeperiod+1):])
    low_diff = np.diff(low[-(timeperiod+1):])
    high_diff[(high_diff<0)] = 0
    low_diff[(low_diff<0)] = 0
#     high_diff[(high_diff<low_diff)] = 0
    low_diff[(high_diff>low_diff)] = 0
    tr = calc_atr_test(high, low, close, timeperiod)*timeperiod
    res = 100 * low_diff.sum() / tr
    return res

def calc_plus_di_test(high, low, close, timeperiod):
    high_diff = np.diff(high[-(timeperiod+1):])
    low_diff = np.diff(low[-(timeperiod+1):])
    high_diff[(high_diff<0)] = 0
    low_diff[(low_diff<0)] = 0
    high_diff[(high_diff<low_diff)] = 0
#     low_diff[(high_diff>low_diff)] = 0
    tr = calc_atr_test(high, low, close, timeperiod)*timeperiod
    res = 100 * high_diff.sum() / tr
    return res

def calc_log_ret_test(close, lag=0):
    return np.log(close[-(1+15*lag)] / close[-(1+15*(lag+1))])

def upper_shadow_15_perc_test(high, close, open_, lag=0):
    if lag:
        return high[-15*(lag+1):-15*lag].max() * 100 / np.maximum(close[-15*lag-1], open_[-15*(lag+1)-1])
    else:
        return high[-15:].max() * 100 / np.maximum(close[-1], open_[-16])

def lower_shadow_15_perc_test(low, close, open_, lag=0):
    if lag:
        return np.minimum(close[-15*lag-1], open_[-15*(lag+1)-1]) * 100 / low[-15*(lag+1):-15*lag].min()
    else:
        return np.minimum(close[-1], open_[-16]) * 100 / low[-15:].min()

In [13]:
def get_features_test(df, open_, high, low, close, volume):
    ### original def features
    res = []
    res.append(calc_sma_diff_test(close, 12, 26))
    res.append(calc_sma_diff_test(close, 12*4*4, 24*4*4))
    res.append(calc_sma_diff_test(close, 12*4*4*4, 24*4*4*4))
    res.append(calc_sma_diff_test(close, 12*4*4*4*4, 24*4*4*4*4))
    res.append(calc_sma_diff_test(volume, 12*4*4, 24*4*4))
    res.append(calc_rsi_test(close, 14*4))
    res.append(calc_natr_test(high, low, close, 14*4*4*4))
    res.append(calc_minus_di_test(high, low, close, 14))
    res.append(calc_minus_di_test(high, low, close, 14*4*4*4))
    res.append(calc_plus_di_test(high, low, close, 14*4))
    res.append(calc_plus_di_test(high, low, close, 14*4*4*4))
    res.append(calc_log_ret_test(close))
    res.append(calc_log_ret_test(close, lag=1))
    res.append(calc_log_ret_test(close, lag=2))
    res.append(upper_shadow_15_perc_test(high, close, open_))
#     df["upper_shadow_15_lag1"] = upper_shadow_15_perc_test(high, close, open_, lag=1)
#     df["upper_shadow_15_lag2"] = upper_shadow_15_perc_test(high, close, open_, lag=2)
    res.append(lower_shadow_15_perc_test(low, close, open_))
#     df["lower_shadow_15_lag1"] = lower_shadow_15_perc_test(low, close, open_, lag=1)
#     df["lower_shadow_15_lag2"] = lower_shadow_15_perc_test(low, close, open_, lag=2)
    res.append(calc_bbands_test(close, 5*4*2))
    res.append(calc_bbands_test(close, 5*4*4))
    res.append(calc_bbands_test(close, 5*4*4*2))
    res.append(calc_bbands_test(close, 5*4*4*4))

    # time features
    res.append(df["datetime"].hour)
    res.append(df["datetime"].minute)
#     df["day_of_week"] = df["datetime"].dayofweek
    res.append(df["Asset_ID"])

    res = np.array(res).reshape([1, -1])
    return res

In [14]:
def fillna_in_ohlcv(df_fillna):
    df_fillna[["Open", "High", "Low", "Close", "Volume"]] = df_fillna[["Open", "High", "Low", "Close", "Volume"]].fillna(method="ffill")
    df_cl = df_fillna[["Close"]]
    all_times = pd.date_range(df_cl.index[0],
                              df_cl.index[-1],
                              freq="min")
    df_cl = df_cl.reindex(all_times, method="pad")

    df_fillna = df_fillna.drop(["Close"], axis=1)
    df_fillna = df_cl.merge(df_fillna,
                            left_index=True,
                            right_index=True,
                            how="left")

    df_fillna[["Count", "Volume"]] = df_fillna[["Count", "Volume"]].fillna(0)
    df_fillna["Open"] = df_fillna["Open"].fillna(df_fillna["Close"])
    df_fillna["High"] = df_fillna["High"].fillna(df_fillna["Close"])
    df_fillna["Low"] = df_fillna["Low"].fillna(df_fillna["Close"])
#     df_fillna["VWAP"] = df_fillna["VWAP"].fillna(df_fillna["Close"])    

    return df_fillna

In [15]:
def get_model_for_asset(df_train, asset_id):
    df_proc = df_train[df_train["Asset_ID"] == asset_id]
    df_proc = fillna_in_ohlcv(df_proc)
    np_open = df_proc.Open.values[-(24*4*4*4*4+50):]
    np_high = df_proc.High.values[-(24*4*4*4*4+50):]
    np_low = df_proc.Low.values[-(24*4*4*4*4+50):]
    np_close = df_proc.Close.values[-(24*4*4*4*4+50):]
    np_volume = df_proc.Volume.values[-(24*4*4*4*4+50):]

    model = pickle.load(open(MODEL / f'model_{asset_id}.sav', 'rb'))
    return model, np_open, np_high, np_low, np_close, np_volume

In [16]:
models = {}
nps_open = {}
nps_high = {}
nps_low = {}
nps_close = {}
nps_volume = {}
for asset_id in weight.keys():
    print(f"Loading model for ID={asset_id:<2}")
    model, np_open, np_high, np_low, np_close, np_volume = get_model_for_asset(df_train, asset_id)
    models[asset_id] = model
    nps_open[asset_id] = np_open
    nps_high[asset_id] = np_high
    nps_low[asset_id] = np_low
    nps_close[asset_id] = np_close
    nps_volume[asset_id] = np_volume
    
del df_train
gc.collect()

Loading model for ID=0 


C:\Users\Eduardo\AppData\Local\Temp\ipykernel_16180\3054822095.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fillna[["Open", "High", "Low", "Close", "Volume"]] = df_fillna[["Open", "High", "Low", "Close", "Volume"]].fillna(method="ffill")


NameError: name 'MODEL' is not defined